In [1]:
import os 
import numpy as np
from utils.utils import *

In [10]:
np.load(os.path.join(fits_dir, 'fits_inversed', 'subj01', 'fits_subj01_train_V1_full_1.npy'))

array([[-3.22538835e-01, -2.50801802e-01,  4.75584835e+00, ...,
        -3.33786011e-06,  3.74755377e-01, -2.23172022e+00],
       [-1.04870893e+00, -1.90814812e-01,  1.09153668e+01, ...,
         1.48057938e-04,  1.06592716e+00, -1.75077947e+00],
       [-3.99650638e-01, -3.03729602e-01,  3.91968059e-01, ...,
         2.19047070e-04,  4.42866712e-01, -2.22065903e+00],
       ...,
       [ 8.73933034e-01, -1.04999999e+00,  3.62233920e-01, ...,
         6.33835793e-04,  8.94521794e-01,  2.44745437e+00],
       [ 1.04974099e+00,  1.79335639e-01,  4.70739379e+00, ...,
         2.54988670e-04,  1.06694257e+00,  1.40159183e+00],
       [ 1.04997256e+00, -1.04992195e+00,  4.26242796e-01, ...,
         3.23355198e-04,  1.06717041e+00,  2.35617039e+00]])

In [5]:
for i in range(5):
    print(i)

0
1
2
3
4


In [31]:
train_path = os.path.join(nsd_dir, "full_brain", "subj03",  f'subj03_betas_list_nativesurface_train_full_0.npy')
test_path = os.path.join(nsd_dir, "full_brain", "subj03",  f'subj03_betas_list_nativesurface_test_full_0.npy')
betas_mask = np.load(os.path.join(nsd_dir, "full_brain", "subj03",  f'subj03_betas_list_nativesurface_train_test_mask_full_4.npy'), allow_pickle=True, mmap_mode='r')
betas_train = np.load(train_path, allow_pickle=True, mmap_mode='r')
betas_test = np.load(test_path, allow_pickle=True, mmap_mode='r')
print(betas_train.shape)
print(betas_test.shape)
betas_mask.shape

(96771, 9411)
(96771, 6234)


(9411,)

In [33]:
train_path = os.path.join(betas_dir, f'subj03_betas_list_nativesurface_train.npy')
test_path = os.path.join(betas_dir,  f'subj03_betas_list_nativesurface_test.npy')
betas_mask = np.load(os.path.join(betas_dir,  f'subj03_betas_list_nativesurface_train_test_mask.npy'), allow_pickle=True, mmap_mode='r')
betas_train = np.load(train_path, allow_pickle=True, mmap_mode='r')
betas_test = np.load(test_path, allow_pickle=True, mmap_mode='r')
print(betas_train.shape)
print(betas_test.shape)
betas_mask.shape

(35914, 9411)
(35914, 6234)


(9411,)

In [36]:
np.load(os.path.join(fits_dir, 'fits_inversed', 'subj01', 'fits_subj01_train_V1_inversed.npy'))

array([[ 0.37231991, -0.6239036 ,  5.08852568, ...,  0.03644979,
         0.70181114,  2.60355987],
       [ 0.41821606, -0.59490903,  6.37151063, ...,  0.03710508,
         0.7272011 ,  2.52886171],
       [ 0.50218017, -0.54680258,  5.6033217 , ...,  0.03313172,
         0.77852532,  2.39870761],
       ...,
       [-0.59113927,  0.18451905,  9.54365055, ..., -0.08575714,
         0.83866704, -1.26823818],
       [-0.56073665, -0.16390155,  9.99575001, ..., -0.02401721,
         0.81752208, -1.8551711 ],
       [-0.67776862,  0.02496036, 10.34656595, ..., -0.05834866,
         0.90182429, -1.5339857 ]])

In [22]:
betas_mask.shape

(10000,)

#### Ben' spseudo code 
%First, calcualte the best fit a model could ever do.

rss=sum((betas_train - betas_test).^2);

%Next, calculate the residual if we explained NO variance

variance=sum((betas_test-mean(betas_test)).^2);

ve_test=1-rss/variance


In [46]:
best_variance = np.zeros((len(subj_list), len(rois.keys())))

for i, subj in enumerate(subj_list):
    train_path = os.path.join(betas_dir, f'{subj}_betas_list_nativesurface_train.npy')
    test_path = os.path.join(betas_dir, f'{subj}_betas_list_nativesurface_test.npy')
    betas_mask = np.load(os.path.join(betas_dir, f'{subj}_betas_list_nativesurface_train_test_mask.npy'), allow_pickle=True)
    if subj == 'subj06' or subj == 'subj08':
        roi_mask = os.path.join(mask_dir, subj, f'short.reduced.nans.{subj}.testrois.npy')
    else:
        roi_mask = os.path.join(mask_dir, subj, f'short.reduced.{subj}.testrois.npy')
    betas_train = np.load(train_path, allow_pickle=True)
    betas_test = np.load(test_path, allow_pickle=True)
    mask = np.load(roi_mask, allow_pickle=True)
    
    betas_train = betas_train[:, betas_mask]
    for j, roi in enumerate(rois.keys()):
    
        rss = np.sum((betas_train[mask == j+1] - betas_test[mask == j+1])**2)
    
       # best_variance[i, j] = np.mean(rss)

        variance = np.sum((betas_test[mask == j+1] - np.mean(betas_test[mask == j+1]))**2) 
        ve_test = 1-rss/variance
        
  
        best_variance[i, j] = ve_test
print(best_variance)                

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
[[-0.03448963 -0.06181324 -0.10216272 -0.17300737 -0.16567516 -0.17079556
  -0.17788923 -0.17636597 -0.21460748 -0.20562375  0.00491673 -0.09029114]
 [ 0.00903153 -0.13262546 -0.11150718 -0.0707922  -0.12834942 -0.15172374
  -0.17125499 -0.14599991 -0.16741943 -0.16575265 -0.07876277 -0.14381754]
 [-0.18467832 -0.22631693 -0.26961887 -0.28681815 -0.22504842 -0.2514205
  -0.25025797 -0.18820858 -0.32271123 -0.29238236 -0.

In [9]:
path = os.path.join(betas_dir, 'subj03_betas_list_nativesurface_train_test_mask.npy')
mask = np.load(path)
mask.shape


(9411,)